In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Analysis of OBD2 data logged from bluetooth adapter

### Car: Mercedes-Benz C-Class 250 T Wagon S204 phase-II 7G-Tronic (2011)

Data from:
https://www.automobile-catalog.com/car/2011/1551800/mercedes-benz_c_250_blueefficiency_t-modell_7g-tronic.html


In [2]:
import pandas as pd
import numpy as np
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
from numpy import inf

#plotly.tools.set_credentials_file(username='maximiliano1985', api_key='gYru09RiRvdlDpIi957O')
#plotly.offline.init_notebook_mode(connected=True)

### Car data

In [3]:
measured_fuel_consumption = [15.06, 16.09, 15.54, 15.29, 15.09, 14.11, 14.95, 14.89, 15.55, 15.74, 16.98, 15.57]

gears = [4.377, 2.859, 1.921, 1.368, 1, 0.82, 0.728]
#gears_overall = [10.81, 7.06, 4.74, 3.38, 2.47, 2.03, 1.8]
final_ratio = 2.47
reverse = 3.416

wheel = {'width_mm':225, 'profile_perc':45, 'diameterRim_inches':17}
wheel['diameterRim_mm'] = wheel['diameterRim_inches']*25.4
wheel['diameterTot_m'] = (wheel['diameterRim_mm'] + 2*(wheel['profile_perc']/100*wheel['width_mm']) )/1000

print("Wheel data", wheel)

print("Gear ratio: ", gears)
print("Final ratio: ", final_ratio)
print("Fuel consumption estimated from tank refills:", measured_fuel_consumption)

Wheel data {'width_mm': 225, 'profile_perc': 45, 'diameterRim_inches': 17, 'diameterRim_mm': 431.79999999999995, 'diameterTot_m': 0.6343}
Gear ratio:  [4.377, 2.859, 1.921, 1.368, 1, 0.82, 0.728]
Final ratio:  2.47
Fuel consumption estimated from tank refills: [15.06, 16.09, 15.54, 15.29, 15.09, 14.11, 14.95, 14.89, 15.55, 15.74, 16.98, 15.57]


In [4]:
df = pd.read_csv('logs/2019-04-12h06_29_02.761616_obdData.log',sep=';')
df['Time_s'] = df['Time_s']-df['Time_s'][0]

df['SPEED'] = df['SPEED'] + 0 #+3 correct for actual speed (TO BE CHEDKED WITH GPS=

Logged data

In [5]:
df

,Time_s,THROTTLE_POS,SPEED,RPM,ENGINE_LOAD,FUEL_LEVEL,COOLANT_TEMP,MAF,ACCELERATOR_POS_D,OIL_TEMP
0,0.000000,48.627451,10,866.00,33.333333,70.980392,24,9.90,21.568627,16
1,0.107598,49.803922,10,866.00,33.333333,70.980392,24,15.71,21.568627,16
2,0.272009,49.803922,10,1063.00,17.647059,70.980392,24,15.71,21.568627,16
3,0.375608,49.803922,10,1063.00,17.647059,70.980392,24,15.71,13.333333,16
4,0.480156,49.803922,10,1063.00,17.647059,70.980392,24,15.71,13.333333,16
5,0.598673,49.803922,10,1017.75,9.803922,70.980392,24,14.71,13.333333,16
6,0.703581,49.803922,10,1017.75,9.803922,70.980392,24,14.71,5.490196,16
7,0.827767,49.803922,10,1017.75,9.803922,70.980392,24,14.71,5.490196,16
8,1.082993,49.803922,10,1017.75,9.803922,70.980392,24,14.71,5.490196,16
9,1.130210,48.627451,10,1017.75,9.803922,70.980392,24,10.28,5.490196,16


### Engine map

Torque data is clipped to 100%, and not really very usefull to assess behavior at high torques.

TODO: BSFC once the instantaneous fuel flow is estimated.

In [6]:
pRpm = go.Scatter(
    x=df['RPM'],#df['Time_s'],
    y=df['ENGINE_LOAD'],
    mode='markers'
)

layout = go.Layout(
    title='Engine performance',
    xaxis=dict( title='Engine speed (RPM)' ),
    yaxis=dict( title='Engine torque (%)' )
)

fig = go.Figure(data=[pRpm], layout=layout)
py.iplot(fig)

PlotlyRequestError: Account limit reached: Your account is limited to creating 25 charts. To continue, you can override or delete existing charts or you can upgrade your account at: https://plot.ly/products/cloud

### Analyze gear

In [ ]:
pGear = [go.Scatter(
    x=df['RPM'],
    y=df['SPEED'],
    mode='markers',
    name = 'Data')]

rpm = np.linspace(0, max(df['RPM'])+100, num=2)

gears_overall = []

for i in range(0,len(gears)):
    wheel_rpm   = rpm/(gears[i]*final_ratio)
    wheel_radps = wheel_rpm*2*np.pi/60
    kmph        = wheel_radps*(wheel['diameterTot_m']/2)*3.6
    gears_overall.append(kmph[1]/rpm[1])
    pGear.append(
        go.Scatter(
            x=rpm,
            y=kmph,
            name = str(i+1)+' gear'
        )
    )

layout = go.Layout(
    title='Powertrain performance',
    xaxis=dict( title='Engine speed (RPM)' ),
    yaxis=dict( title='Vehicle speed (km/h)' )
)

#fig = go.Figure(data=pGear, layout=layout)
#py.iplot(fig)

In [ ]:
RPM_idle = 650

gears_overall
dist_thrs = 5 # km/h

indxs = [[], [], [], [], [], [], []]
for i in range(0, len(df['SPEED'])):
    x = df['RPM'][i]
    y = df['SPEED'][i]
    
    distances = []
    indx_min = 0
    dist_min = 1e6
    if x <= RPM_idle:
        indx_min = 0
    else:
            
        for j in range(0, len(gears_overall)):
            d = (abs(gears_overall[j]*x - 1*y )/np.sqrt(gears_overall[j]**2+1))
            if d < dist_min:
                indx_min = j
                dist_min = d
        if dist_min < dist_thrs:
            indxs[indx_min].append(i)   

In [ ]:
df_1 = df.loc[indxs[0], :]
df_2 = df.loc[indxs[1], :]
df_3 = df.loc[indxs[2], :]
df_4 = df.loc[indxs[3], :]
df_5 = df.loc[indxs[4], :]
df_6 = df.loc[indxs[5], :]
df_7 = df.loc[indxs[6], :]

print('Statistics')
for i in range(0, len(indxs)):
    percTimeInGear = np.round(len(df.loc[indxs[i], :])/len(df)*100)
    avrgTorqueInGear = np.mean(df['ENGINE_LOAD'][indxs[i]])
    print('\tgear ', i+1, ': percTimeInGear', percTimeInGear, '%, avg.Trq ',  round(avrgTorqueInGear), '%')


_Plot on the left_: logged engine speed and vehicle speed it is possible to assess the gear ratios of the 7-gear automatic transmission. The match between measured ratios and nominal ones from datasheet is good, meaning that the error of the measured speed is small.

_Plot on the right_: by plotting the vehicle speed vs the engine load and colouring the points based on estimated gear it should be possible to reverse engineer the gear shifting strategy. However, the data is very noisy, mainly due to the fact that this automatic transmission has a torque converter, hence a complex behavior.

In [ ]:
colors = ['rgba(200, 0, 0, 0.2)',
        'rgba(150, 50, 0, 0.2)',
        'rgba(100, 100, 0, 0.2)',
        'rgba(50, 150, 100, 0.2)',
        'rgba(0, 200, 0, 0.2)',
        'rgba(0, 150, 50, 0.2)',
        'rgba(0, 100, 100, 0.2)',]

colors = ['rgba(200, 0, 0, 0.2)',
        'rgba(0, 200, 0, 0.2)',
        'rgba(0, 0, 200, 0.2)',
        'rgba(150, 50, 0, 0.2)',
        'rgba(0, 50, 100, 0.2)',
        'rgba(100, 0, 100, 0.2)',
        'rgba(75, 75, 75, 0.2)',]
pGearColors   = []
pEngineColors = []
fig = tools.make_subplots(rows=1, cols=2)

for i in range(0,len(gears)):
    pGearColors = (
        go.Scatter(
            x=rpm,
            y=gears_overall[i]*rpm,
            name = str(i+1)+' gear',
            mode='lines',
            line = dict(color = colors[i]),
            showlegend=False
        )
    )
    fig.append_trace(pGearColors, 1, 1)
    pGearColors = (
        go.Scatter(
            x= df['RPM'][indxs[i]],
            y= df['SPEED'][indxs[i]],
            #name = str(i+1)+' gear',
            mode='markers',
            marker = dict(color = colors[i]),
            showlegend=False
        )
    )
    fig.append_trace(pGearColors, 1, 1)
    
    pEngineColors = (
        go.Scatter(
            x= df['SPEED'][indxs[i]],
            y=df['ENGINE_LOAD'],
            #name = str(i+1)+' gear',
            mode='markers',
            marker = dict(color = colors[i]),
            showlegend=False
        )
    )
    fig.append_trace(pEngineColors, 1, 2)

#fig.append_trace(pGearColors, 1, 1)
#fig.append_trace(pEngineColors, 1, 2)

fig['layout']['xaxis1'].update(title='Engine (RPM)')
fig['layout']['xaxis2'].update(title='Vehicle speed (km/h)')
fig['layout']['yaxis1'].update(title='Vehicle speed (km/h)')
fig['layout']['yaxis2'].update(title='Engine load (%)')
fig['layout'].update(height=500, width=1000)

py.iplot(fig, filename='[pGearColors]')

# Classifier to reverse engineer the gear shifting strategy

### Estimate fuel consumption

Fuel consumption is estimated by assuming:
A2D = 14.6 air-to-diesel stechiometric ratio
DD = density of diesel = 0.832 # kg/litre


Measured values:  
__MAF__: (grams/sec) Mass Air Flow  
__S__  : (km/h) vehicle longitudinal speed


The fuel mass flow MFF (grams/sec) is:  
__MFF__ = MAF/A2D  


Fuel volumetric flow (litre/h):  
__VFF__ = MFF/(DD\*1000)\*3600


Fuel consumption KMPL (km/litre):  
__KMPL__ = S/VFF


TODO: fuel consumption seems to be over estimated. Correction factor needed, because fuel tank refill statistics state that the average fuel consumption is 15 km/l.

In [ ]:
df['MAF'] = df['MAF']
pMAF = go.Scatter(
    x=df['Time_s'],
    y=df['MAF'],
    name = 'MAF'
    #mode='markers+text'
)

pSPEED = go.Scatter(
    x=df['Time_s'],
    y=df['SPEED'],
    name = 'SPEED'
    #mode='markers+text'
)

layout = go.Layout(
    #title='Powertrain performance',
    xaxis=dict( title='Time (s)' ),
    yaxis=dict( title='' )
)

fig = go.Figure(data=pGear, layout=layout)
py.iplot([pSPEED, pMAF])

In [ ]:
# Natural gas: 17.2
# Gasoline: 14.7
# Propane: 15.5
# Ethanol: 9
# Methanol: 6.4
# Hydrogen: 34
# Diesel: 14.6

AIR2DIESEL_RATIO = 14.6
DENSITYDIESEL    = 0.832 # kg/litre !!! ADJUST THIS VALUE IF NEEDED

df['MFF']  = df['MAF']/AIR2DIESEL_RATIO # (grams/sec) Mass Fuel Flow
df['VFF']  = df['MFF']/(DENSITYDIESEL*1000)*3600 # (litres/h) Volume Fuel Flow


df['KMPL'] = df['SPEED']/df['VFF'] # (km/litre) Fuel consumption
print("Averaged fuel consumption while moving: ", np.mean(df['KMPL']), " km/l")

In [ ]:
pMFF = go.Scatter(
    x=df['Time_s'],
    y=df['MFF'],
    name = 'MFF'
    #mode='markers+text'
)
pKMPL = go.Scatter(
    x=df['Time_s'],
    y=df['KMPL'],
    name = 'KMPL'
    #mode='markers+text'
)
pVFF = go.Scatter(
    x=df['Time_s'],
    y=df['VFF'],
    name = 'VFF'
    #mode='markers+text'
)
py.iplot([pKMPL,pKMPL2], filename='[pKMPL]')

In [ ]:
df.mean()